![Aprendé a minar y analizar datos con IA localmente](assets/header.png)

Antes que nada, no te olvides de crear el entorno virtual!
```sh
uv sync
```

In [1]:
import workshop

In [2]:
dataset_path = workshop.dataset.argendata_topicos.download()

  0%|          | 0/20 [00:00<?, ?it/s]

In [3]:
from pathlib import Path

In [4]:
dataset_files = Path('./data/argendata_topicos/processed').glob('**/*.md')
dataset_files = list(dataset_files)

## Extracción de _chunks_

In [5]:
from workshop.utils.text_splitter import RecursiveCharacterSplitter

---

In [6]:
splitter_ejemplo = RecursiveCharacterSplitter(
    chunk_size=20, 
    chunk_overlap=10
)

texto_largo_ejemplo = \
"""
Lorem ipsum dolor sit amet, consectetur adipiscing elit. 
"""

chunks_ejemplo = splitter_ejemplo.split_text(texto_largo_ejemplo)

for chunk in chunks_ejemplo:
    print(repr(chunk))

'Lorem ipsum dolor'
'dolor sit amet,'
'amet, consectetur'
'adipiscing elit.'


---

In [7]:
splitter = RecursiveCharacterSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)

In [8]:
from workshop.utils.common import Metadata
from uuid import uuid4

In [9]:
ids = list[str]()
chunks = list[str]()
metadatas = list[dict]()

for file in dataset_files:
    file = Path(file)
    
    for chunk in splitter.split_text(file.read_text(encoding='utf8')):
        ids.append(str(uuid4()))
        
        chunks.append(chunk)

        path = str(file.resolve())
        metadata = Metadata(path=path)
        metadata = metadata.model_dump(mode='json', exclude_none=True)
        metadatas.append(metadata)

In [10]:
print(f'{ids[0]=}')
print(f'{metadatas[0]=}')
print(f'{chunks[0]=}')

ids[0]='18345eb0-3984-48b9-a309-dfbdb07d72aa'
metadatas[0]={'path': 'C:\\Users\\Joan\\Coding\\joangq\\workshop-media-party\\data\\argendata_topicos\\processed\\agroindustria.md'}
chunks[0]='## Actividad agropecuaria en Argentina\n\n### La producción agropecuaria se multiplicó por más de 6 veces en el último siglo, aunque su importancia relativa en la economía argentina ha disminuido en el último tiempo\n\nPNG [CSV](https://raw.githubusercontent.com/argendatafundar/data/main/AGROPE/pbi_agro.csv) Historia reciente del PIB agropecuario Evolución del PIB agropecuario en Argentina (en millones de pesos constantes de 2004), 1875-2022 Fuente de datos: Dos siglos de economía argentina, Fundación Norte y Sur; Agregados macroeconómicos, INDEC.\n\nDesde fines del siglo XIX, la producción agropecuaria, se expandió sostenidamente. Así, en la década de 2010 fue 52% mayor a la de 1990, 178% mayor a la de 1960, 574% mayor a la de 1910 y 2290% mayor a la de 1880.'


## Vectorización

⚠️ En esta sección vamos a empezar a utilizar modelos locales. Mucho del procesamiento _suele_ hacerse en la GPU usando CUDA. 

Instalar CUDA suele ser problemático, así que a modo de ejemplo puede usarse el CPU.

In [11]:
from workshop.utils.sentence_transformers import SentenceTransformer

In [12]:
modelo_embeddings = SentenceTransformer(
    model_name_or_path="sentence-transformers/all-mpnet-base-v2",
    device="cuda", # "cpu"
    backend='torch',
)

---

In [13]:
encodings_ejemplo = modelo_embeddings.encode(chunks_ejemplo)

for i, encoding in enumerate(encodings_ejemplo):
    print(chunks_ejemplo[i])
    for value in encoding:
        print(f'{value:.3f}', end=' ')
    print('\n')

Lorem ipsum dolor
0.002 0.020 -0.028 0.061 -0.071 0.017 -0.002 0.074 0.043 0.004 0.055 0.010 0.004 -0.019 0.013 -0.046 0.010 0.015 -0.015 -0.033 -0.018 0.033 0.013 -0.002 0.063 -0.052 -0.019 0.065 -0.007 -0.025 0.036 -0.017 0.004 0.001 0.000 -0.034 0.016 0.016 0.012 -0.060 -0.016 0.092 0.028 -0.037 -0.031 0.089 -0.034 -0.012 -0.042 0.052 0.038 -0.041 0.014 -0.026 0.132 0.025 -0.002 -0.008 -0.004 0.049 0.036 -0.023 -0.014 -0.011 -0.083 -0.025 0.041 0.004 0.032 -0.026 0.051 -0.014 0.021 0.129 0.011 -0.017 -0.016 0.004 -0.024 -0.041 0.042 0.036 0.018 0.032 0.018 0.015 0.023 -0.010 0.032 -0.053 -0.034 0.003 0.005 0.041 0.014 -0.047 -0.031 -0.011 0.024 -0.012 -0.034 0.017 -0.025 0.047 -0.038 -0.045 0.072 0.009 -0.019 0.019 0.022 0.028 0.012 0.069 -0.012 0.016 -0.032 -0.021 -0.033 0.005 -0.019 -0.008 0.063 -0.002 -0.008 -0.026 0.044 0.024 -0.037 0.009 0.019 0.077 -0.036 -0.019 0.056 0.044 -0.014 -0.059 -0.075 0.025 0.057 -0.044 -0.016 0.001 -0.012 0.007 0.051 -0.114 -0.016 0.025 0.027 0.016 

---

In [14]:
vectores = modelo_embeddings.encode(chunks, show_progress_bar=True)

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [15]:
import chromadb

In [16]:
db = chromadb.PersistentClient(path="chroma")

In [17]:
coleccion = db.create_collection("argendata_topicos")

In [18]:
coleccion.add(
    ids=ids,
    documents=chunks,
    embeddings=vectores,
    metadatas=metadatas,
)

### Visualización

💡 Esto no es necesario para el funcionamiento, pero nos ayudará a comprender mejor el proceso.

In [19]:
# https://umap-learn.readthedocs.io/en/latest/
from workshop.utils.umap import UMAP

import numpy as np
from tqdm import tqdm

In [20]:
umap = UMAP(
    random_state=0,
    transform_seed=0,
)

In [21]:
# Ajustamos el modelo sobre nuestros datos
umap_transform = umap.fit(vectores)

c:\Users\Joan\Coding\joangq\workshop-media-party\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [22]:
result = np.empty((len(chunks), 2))
for i, chunk in enumerate(tqdm(chunks)):
    result[i] = umap_transform.transform([vectores[i]]) # type: ignore[unsupported-operation]

100%|██████████| 1083/1083 [00:07<00:00, 141.62it/s]


In [23]:
for i, metadata in enumerate(metadatas):
    metadata['projection_x'] = result[i][0]
    metadata['projection_y'] = result[i][1]

In [24]:
print(ids[0])
print(metadatas[0]['projection_x'])
print(metadatas[0]['projection_y'])
# print(chunks[0]) # para ver el contenido del chunk

18345eb0-3984-48b9-a309-dfbdb07d72aa
1.520433783531189
8.676289558410645


In [25]:
coleccion.update(
    ids=ids,
    metadatas=metadatas,
)

In [26]:
import altair as alt
import polars as pl

In [27]:
df = pl.DataFrame([
    {
        'id': id,
        'chunk': f'{chunk[:100]}...',
        'path': metadata['path'],
        'topico': Path(metadata['path']).stem.capitalize().replace('-', ' '),
        'projection_x': metadata['projection_x'],
        'projection_y': metadata['projection_y'],
    }
    for id, chunk, _, metadata in zip(ids, chunks, vectores, metadatas)
])

In [28]:
df

id,chunk,path,topico,projection_x,projection_y
str,str,str,str,f64,f64
"""18345eb0-3984-48b9-a309-dfbdb0…","""## Actividad agropecuaria en A…","""C:\Users\Joan\Coding\joangq\wo…","""Agroindustria""",1.520434,8.67629
"""64c2eb27-9815-4797-9b7a-a57cd5…","""PNG [CSV](https://raw.githubus…","""C:\Users\Joan\Coding\joangq\wo…","""Agroindustria""",1.345654,8.291393
"""097c4467-f01a-4730-87fb-afbbad…","""### La agricultura representa …","""C:\Users\Joan\Coding\joangq\wo…","""Agroindustria""",1.61498,8.798896
"""74422c23-4fa4-41e3-8c1f-2c9513…","""Dentro de la agricultura se in…","""C:\Users\Joan\Coding\joangq\wo…","""Agroindustria""",1.389177,9.385313
"""c23beab5-fd73-46c1-8352-f24d63…","""### La reducción en la importa…","""C:\Users\Joan\Coding\joangq\wo…","""Agroindustria""",1.57011,8.67543
…,…,…,…,…,…
"""06a1382d-44cd-44bb-8f18-85dc63…","""PNG [CSV](https://raw.githubus…","""C:\Users\Joan\Coding\joangq\wo…","""Transicion energetica""",8.757243,5.243263
"""9621af2e-48db-402e-b50b-95bcf1…","""Desde 1965 se ve una disminuci…","""C:\Users\Joan\Coding\joangq\wo…","""Transicion energetica""",8.621856,6.047315
"""9fcdb13a-0242-4f10-ae8e-82f258…","""PNG [CSV](https://raw.githubus…","""C:\Users\Joan\Coding\joangq\wo…","""Transicion energetica""",8.828772,5.416541


In [33]:
chart = alt.Chart(df).mark_circle(
    size=50,
).encode(
    x='projection_x',
    y='projection_y',
    tooltip=['id', 'topico', 'chunk'],
    color=alt.Color('topico', scale=alt.Scale(scheme='tableau20')),
).properties(
    title='Proyección de los embeddings',
    width=1000,
    height=800,
).interactive()

chart.show()

alt.Chart(...)

In [34]:
def k_nearest_neighbors(
    query: str,
    collection: chromadb.Collection,
    model: SentenceTransformer,
    k: int = 10,
    include: list[str] = ["metadatas", "documents"],
):
    query_embedding = model.encode([query])
    
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=k,
        include=include,
    )

    include = ['ids', *include]
    result = (
        results[key][0]
        for key in include
    )

    result = zip(result)
    result = [x[0] for x in result]

    return result

In [39]:
# query = "¿Cuánto más alta es hoy la temperatura promedio en Argentina respecto de 1940-1969?"
# query = "¿Cuántos países producen energía nuclear y Argentina está entre ellos?"
query = "¿Superará Argentina los 50 millones de habitantes según las proyecciones?"

ids, metadatas, documents = k_nearest_neighbors(
    query, 
    coleccion, 
    modelo_embeddings,
    include = ['metadatas', 'documents'],
    k = 10
)

In [41]:
new_points = alt.Chart(
    pl.DataFrame([
        {
            'id': id,
            'chunk': chunk,
            # path: metadata['path'],
            'topico': Path(metadata['path']).stem.capitalize().replace('-', ' '),
            'projection_x': metadata['projection_x'],
            'projection_y': metadata['projection_y'],
        }
        for id, chunk, metadata in zip(ids, documents, metadatas)
    ])
).mark_point(
    size=100,
    opacity=1,
    stroke='black',
    color='black',
    shape='circle',
).encode(
    x='projection_x',
    y='projection_y',
    tooltip=['id', 'topico', 'chunk'],
    color=alt.Color('topico', scale=alt.Scale(scheme='tableau20')),
).properties(
    title=f'Query: {query}',
    width=1000,
    height=800,
)

chart = chart.properties(title='')
new_chart = chart+new_points

new_chart.show()

alt.LayerChart(...)

In [50]:
from workshop.agent import ContextAgent
from workshop.utils import mostrar_respuesta

Tokenizador online: https://tiktokenizer.vercel.app/?model=meta-llama%2FMeta-Llama-3-8B

Ollama: https://ollama.com/search

⚠️ Antes de ejecutar esto, asegurate de tener Ollama instalado y corriendo.

Desde la consola, para inciar el servidor podés hacer
```sh
ollama serve
```

Luego, el modelo que vaya a usarse se puede instalar explícitamente con
```sh
ollama pull [MODELO]

# Por ejemplo:
ollama pull llama3.2:3b-instruct-q8_0
```

⚠️ Utilizar modelos locales utiliza muchos recursos. Asegurate de tener bien venitlada la computadora.

In [ ]:
agent = ContextAgent(
    'ollama/llama3.2:3b-instruct-q8_0', 
    # 'ollama/phi3:3.8b-mini-128k-instruct-q6_K',
    # 'ollama/deepseek-r1:1.5b-qwen-distill-fp16',
    'Clearly answer the questions provided by the user, using the context. Always answer in spanish, and as conciesly as possible.', 
    '<question>$prompt</question> <context>En Argendata (argendata.fund.ar) se dice esto: $context</context>', 
    coleccion, 
    modelo_embeddings
)

In [52]:
respuesta = agent.completion("¿Cuál es la principal fuente limpia en la electricidad de Argentina?")
mostrar_respuesta(respuesta)

El análisis de la matriz energética argentina revela una evolución compleja en el uso de las diferentes fuentes de energía primaria, con un cambio significativo hacia la hidroelectricidad y el gas natural, especialmente en comparación con otros países de América Latina. La participación del petróleo disminuyó, mientras que el papel de las renovables, incluidas la energía eólica, es menor que el promedio mundial.

En cuanto al desarrollo económico, la región patagónica ha experimentado un crecimiento significativo en su contribución al PIB argentino, pasando de tener solo 0,5% de participación en 1895 a más del 9% actualmente. Provincias como Neuquén han sido particularmente impactadas, ganando peso en el PIB, mientras que la Ciudad Autónoma de Buenos Aires ha visto una disminución en su participación.

La comparación internacional muestra a Argentina con niveles de ingresos laborales medianos-alto dentro de América Latina. Los desafíos enfrentados por los energéticos sostenibles en Argentina incluyen la intermitencia de la producción solar, necesidades de espacio y almacenamiento eficiente, desarrollo de tecnologías para el manejo de residuos y la gestión de recursos minerales.

En conclusión, la matriz energética argentina se caracteriza por una fuerte participación de la hidroelectricidad y del gas natural, con una menor presencia del petróleo. La región patagónica ha experimentado un crecimiento económico significativo, mientras que la Ciudad Autónoma de Buenos Aires ha visto disminuciones en su contribución al PIB.

In [53]:
respuesta = agent.completion("¿Qué puedes decirme de la producción de energía nuclear en Argentina?")
mostrar_respuesta(respuesta)

Lo siento, pero parece que hay un error en la presentación. La respuesta proporcionada no está relacionada con la pregunta planteada. Sin embargo, puedo ayudarte a reorganizarla de acuerdo con tu solicitud.

**Las energías limpias**

Las energías limpias son una fuente fundamental para la transición energética hacia una matriz menos contaminante. En Argentina y en el mundo, las energías limpias ocupan un porcentaje significativo de la matriz primaria energética.

### La energía hidroeléctrica en Argentina

La energía hidroeléctrica es la principal fuente de energía limpia en Argentina, representando el 47% del total de las energías bajas en carbono. Su participación en la matriz eléctrica argentina es del 40,5%, similar a la media mundial.

### La energía hidroeléctrica en el mundo

En el mundo, la energía hidroeléctrica también es una fuente importante de energía limpia, representando el 36% del total de las energías bajas en carbono. Su participación en la matriz eléctrica mundial es del 39,3%.

### Comparación con otras fuentes de energía limpia

En Sudamérica, la energía hidroeléctrica desempeña un papel destacado, especialmente en Brasil (60,5%), Colombia (62,5%) y Paraguay (100%).

La participación de las energías limpias en la matriz eléctrica argentina es similar a la media mundial, pero mucho menor al promedio regional. La generación hidroeléctrica en Argentina cobró mayor énfasis a partir de la década de 1970.

**Conclusión**

En resumen, las energías limpias, especialmente la energía hidroeléctrica, juegan un papel crucial en la transición energética. En Argentina y en el mundo, su participación es significativa, pero hay una brecha con respecto a otros sectores regionales como Sudamérica.

In [54]:
respuesta = agent.completion("¿Cómo evolucionó el PIB per cápita argentino desde 1975?")
mostrar_respuesta(respuesta)

Este es el texto proporcionado:

En Argentina, desde finales del siglo XX y durante todo el siglo XXI, se observa un patrón de crecimiento sostenido con una mejora del Índice de Desarrollo Humano (IDH) que va asociada a una evolución positiva en varios indicadores sociales. Estos incluyen tanto la suba simultánea del ingreso per cápita, los años de escolarización de la población y la esperanza de vida como el aumento del acceso a servicios básicos.

El crecimiento del PIB argentino fue fuerte hasta 2002, con un importante crecimiento del sector de la pesca. Hasta fines de la década de 1970, las pesquerías tenían una participación en el PIB del 0,1%. Sin embargo, el aumento de las capturas marinas desde mediados de esta década hizo que este sector ganara participación sostenida en el PIB. En 2002, el sector alcanzó su pico con un 0,48% del PIB. 

Es posible dimensionar la relevancia económica de los SBC a partir de su peso relativo en la facturación total de las empresas argentinas. En 2021, los SBC dieron cuenta del 3,2% de las ventas totales de las empresas argentinas y aumentaron 0,6 puntos porcentuales desde 2014.

Los siguientes puntos destacan aspectos relevantes sobre el crecimiento económico en Argentina durante este período:

*   **Mejora del IDH**: Durante todo el siglo XXI, Argentina ha observado un patrón de crecimiento sostenido y mejora del Índice de Desarrollo Humano (IDH), asociada a una evolución positiva en diversos indicadores sociales como el ingreso per cápita, años de escolarización de la población y esperanza de vida.
*   **Pobre desempeño del crecimiento económico**: A partir de 2011, el crecimiento económico argentino se ralentizó significativamente, lo que se refleja en una suba lenta del IDH, principalmente debida al pobre desempeño del ingreso nacional bruto (INB) per cápita.
*   **Impacto de la pandemia**: La pandemia tuvo un impacto negativo considerablemente en el IDH argentino, con un retroceso en la esperanza de vida y una severa recesión que redujo el INB per cápita.

In [57]:
respuesta = agent.completion("¿Qué lugar ocupan hoy el oro, la plata y el litio en las exportaciones mineras?")
mostrar_respuesta(respuesta)

La respuesta proporcionada no sigue el formato solicitado. A continuación, se presenta la información solicitada en un formato adecuado:

**¿Qué productos exporta e importa la agroindustria argentina?**

*   Los principales productos exportados por Argentina son:
    *   Soja: 34,9% de las exportaciones mundiales.
    *   Pellets de soja: 22,2% del mercado mundial.
    *   Maíz: 12% de las exportaciones globales.
    *   Aceite de girasol y cebada: cuarto en el mundo (2023).
    *   Langostinos congelados: quinto lugar en la exportación mundial (2023).

*   Los principales productos importados por Argentina son:
    *   Oliba virgen extra.
    *   Aceite de oliva.
    *   Maíz.

**¿Qué son los principales minerales que exporta Argentina?**

Los principales minerales que exporta Argentina en la actualidad son:

1.  Oro
2.  Plata
3.  Litio

Además, el país también exporta otros minerales como cobre y molibdeno.

**¿Cuál es el complejo pesquero más importante?**

En 2024, el complejo pesquero exportó 2.007 millones de dólares, representando el 0,3% del PIB argentino y el 2,5% de las exportaciones totales.

**¿Qué es la exportación industrial ligada al complejo sojero?**

La exportación industrial ligada al complejo sojero incluye productos como pellets de soja, aceite de soja, poroto de soja para consumo y otros productos relacionados con el cultivo y procesamiento del soya.

In [59]:
respuesta = agent.completion("¿Cómo se ubica Argentina en términos de los índices de desigualdad en la región?")
mostrar_respuesta(respuesta)

El IDH-A (Índice de Desarrollo Humano Aumentado) de Argentina ha mostrado una tendencia de crecimiento en el último siglo y medio, aunque con algunas fluctuaciones. Aquí hay algunos datos clave sobre la evolución del IDH-A de Argentina:

*   En 1870, el IDH-A de Argentina era de alrededor de 0,35, mientras que la media regional era de 0,32.
*   En 2020, el IDH-A de Argentina fue de 0,78, lo que significa un aumento del 121% en relación con el valor inicial.
*   Entre 1870 y 1929, el mayor IDH-A de la región lo tuvo Uruguay, mientras que Argentina ocupaba el segundo lugar. Sin embargo, en los últimos años, Argentina ha sido uno de los países más desarrollados de la región.
*   El crecimiento del IDH-A de Argentina se debe principalmente a progresos en tres de sus cuatro dimensiones:
    *   PIB per cápita: multiplicado por casi 6 veces entre 1870 y 2020.
    *   Esperanza de vida: aumentó de 32 años a 76 años.
    *   Años de escolarización: pasaron de 1,5 a 11,1 años.
*   El índice de democracia liberal también ha mejorado significativamente, con un aumento del 120% en el mismo período.

In [61]:
respuesta = agent.completion("¿Cómo se ubica Argentina en materia de desigualdad en la región?")
mostrar_respuesta(respuesta)

El informe proporcionado presenta información sobre el valor agregado bruto (VAB) de Argentina, la composición de las exportaciones de servicios y los ingresos laborales en perspectiva internacional. A continuación, se presentan las conclusiones clave:

1. **Valor agregado bruto**: La economía argentina ha experimentado un crecimiento del VAB, aunque con variaciones a lo largo del tiempo. El informe menciona que el país ha tenido niveles de ingresos laborales medio-altos en relación con América Latina.
2. **Composición de las exportaciones de servicios**: Las exportaciones de servicios en Argentina han evolucionado, con viajes como principal componente en las últimas décadas. Sin embargo, este sector ha perdido protagonismo, y nuevos sectores basados en el conocimiento han impulsado el crecimiento económico.
3. **Ingresos laborales en perspectiva internacional**: Los ingresos laborales de Argentina se encuentran entre los medianos de la región América Latina, según las definiciones productiva y legal. Aunque ha tenido un desempeño moderado en comparación con otros países, la situación ha empeorado en las últimas décadas, especialmente cuando se considera la definición legal.
4. **Informalidad laboral**: El informe destaca que Argentina ha tenido una tasa de informalidad laboral significativa, aunque ha disminuido en comparación con otros países de la región. La situación es más preocupante cuando se considera la definición productiva, ya que el país se encuentra entre los cuatro de menores tasas de informalidad.

En resumen, el informe presenta un panorama económico complejo y desafiante para Argentina. Aunque el crecimiento del VAB y las exportaciones de servicios han sido positivos, la situación laboral sigue siendo una preocupación, especialmente en términos de informalidad y desempleo. Para abordar estos desafíos, es fundamental implementar políticas públicas que fomenten la productividad, la participación laboral y el bienestar social, así como aprovechar al máximo la oportunidad del bono demográfico actual para impulsar el crecimiento económico sostenible.